In [ ]:
# This is an implementation of simpilcial homology computation.
# Written around June 4, 2020.

In [414]:
import itertools
import functools
import operator
import numpy as np
import pandas as pd
import sympy as sp

In [469]:
def simplicial_complex(*args):
    result = set()
    for simplex in args:
        for d in range(len(simplex) + 1):
            result.update(tuple(sorted(e)) for e in itertools.combinations(simplex, d))
    return list(sorted(result, key=lambda simplex: (len(simplex), simplex)))

In [433]:
def _delta_coeff(edge, simplex):
    for i in range(len(simplex)):
        if simplex[:i] + simplex[i + 1:] == edge:
            return (-1) ** i
    return 0

def delta(sc):
    if not sc:
        return np.empty(shape=(0, 0), dtype=int)
    res = [[_delta_coeff(sc[i], sc[j]) for j in range(len(sc))] for i in range(len(sc))]
    return np.array(res, dtype=int)

In [53]:
m = delta(c)

In [483]:
def _permutation_matrix(n, s, t):
    result = np.identity(n)
    result[[s, t]] = result[[t, s]]
    return result


def _addition_matrix(n, s, t, k):
    result = np.identity(n)
    result[s, t] = k
    return result


def _to_smith_normal_form(mat, i_s, j_s, a, b):
    n = len(mat)

    if not i_s.size or not j_s.size:
        return

    i0 = i_s[0]
    j0 = j_s[0]

    if np.count_nonzero(mat[np.ix_(i_s, j_s)]) == 0:
        return

    g = np.gcd.reduce(mat[np.ix_(i_s, j_s)], axis=None)
    for i, j in itertools.product(i_s, j_s):
        if mat[i, j] in {g, -g}:
            mat[:] = _permutation_matrix(n, i0, i) @ mat @ _permutation_matrix(n, j0, j)
            a[:] = a @ _permutation_matrix(n, i0, i)
            b[:] = _permutation_matrix(n, j0, j) @ b
            break
    else:
        raise NotImplementedError()
        

    for i in i_s[1:]:
        k = mat[i, j0] // mat[i0, j0]
        mat[:] = _addition_matrix(n, i, i0, -k) @ mat
        a[:] = a @ _addition_matrix(n, i, i0, k)

    for j in j_s[1:]:
        k = mat[i0, j] // mat[i0, j0]
        mat[:] = mat @ _addition_matrix(n, j0, j, -k)
        b[:] = _addition_matrix(n, j0, j, k) @ b

    _to_smith_normal_form(mat, i_s[1:], j_s[1:], a, b)


def smith_normal_form(matrix, minors=None):
    mat = np.copy(matrix)
    n = len(mat)

    if minors is None:
        minors = [(np.arange(n), np.arange(n))]
    else:
        minors = [(np.array(i_s), np.array(j_s)) for i_s, j_s in minors]

    a = np.identity(n, dtype=int)
    b = np.identity(n, dtype=int)

    for i_s, j_s in minors:
        _to_smith_normal_form(mat, i_s, j_s, a, b)
    assert (a @ mat @ b == matrix).all()
    return mat, a, b
    

def list_minors(sc):
    result = []
    if not sc:
        return result
    max_d = max(len(s) for s in sc) - 1
    for d in range(max_d, -1, -1):
        this_slice = [i for i in range(len(sc)) if len(sc[i]) == d + 1]
        prev_slice = [i for i in range(len(sc)) if len(sc[i]) == d]
        result.append((prev_slice, this_slice))
    return result

def inv(mat):
    return np.linalg.inv(mat).astype(int)


def delta_normal_form(sc):
    om = delta(sc)
    m = om
    n = len(m)
    minors = list_minors(sc)

    a = np.identity(n, dtype=int)
    b = np.identity(n, dtype=int)

    for minor in minors:
        m, a_1, b_1 = smith_normal_form(m, [minor])
        assert (a_1 @ b_1 == b_1 @ a_1).all()
        a = a @ a_1
        b = b_1 @ b
        assert (a @ m @ b == om).all()
        
        m = b_1 @ m @ a_1
        a = a @ inv(b_1)
        b = inv(a_1) @ b
        assert (a @ m @ b == om).all()
        # break
    return m, a, b


def repr_simplex(simplex):
    if not simplex:
        return sp.sympify(1)
    return sp.Symbol(''.join(simplex))


def repr_k_simplex(k, simplex):
    return k * simplex


def repr_chain(sc, chain):
    return sum(repr_k_simplex(k, repr_simplex(sc[i])) for i, k in enumerate(chain) if k != 0)


def format_basis(sc, basis):
    result = []
    for j, v in enumerate(basis.T):
        result.append(repr_chain(sc, v))
    return result


f = sp.Function("Factor")
ideal = sp.Function("Span")

def display_delta(sc):
    m, a, b = delta_normal_form(c)

    a_basis = format_basis(sc, a)
    b_basis = format_basis(sc, np.linalg.inv(b).astype(int))

    df = pd.DataFrame(m, index=a_basis, columns=b_basis)
    styled = df.style
    for minor in list_minors(sc):
        styled = styled.set_properties(**{'background-color': '#CCFFDD'}, subset=(df.iloc[minor].index, df.iloc[minor].T.index))
    display(styled)

    homo = []
    for col, chain in zip(m.T, b_basis):
        if np.count_nonzero(col) == 0:
            row = m[a_basis.index(chain)]
            k = max(row, key=lambda x: abs(x))
            if k not in {-1, 1}:
                homo.append(f(ideal(chain), k * ideal(chain)))
                
    display(functools.reduce(operator.mul, homo, sp.sympify(1)))

c = simplicial_complex({'a','b','d'}, {'b', 'c'}, {'c', 'd'}, {'e'})
# c = simplicial_complex(
#     {'0', '1', '2'},
#     {'0', '2', '4'},
#     {'1', '2', '5'},
#     {'0', '1', '3'},
#     {'2', '3', '4'},
#     {'2', '3', '5'},
#     {'1', '4', '5'},
#     {'1', '4', '3'},
#     {'0', '5', '4'},
#     {'0', '5', '3'},
# )
display_delta(c)

,1,d,b - c,c - d,a - d,-d + e,ad,bc,-bc + bd,ab - ad + bd,bc - bd + cd,abd
1,0,1,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0,0,0
b - c,0,0,0,0,0,0,0,-1,0,0,0,0
c - d,0,0,0,0,0,0,0,0,-1,0,0,0
a - d,0,0,0,0,0,0,-1,0,0,0,0,0
-d + e,0,0,0,0,0,0,0,0,0,0,0,0
ad,0,0,0,0,0,0,0,0,0,0,0,0
bc,0,0,0,0,0,0,0,0,0,0,0,0
-bc + bd,0,0,0,0,0,0,0,0,0,0,0,0
ab - ad + bd,0,0,0,0,0,0,0,0,0,0,0,1


Factor(Span(-d + e), 0)*Factor(Span(bc - bd + cd), 0)